# Stylemap Prototype (legacy)

In [20]:
"""mplchart stylemap (experimental)"""

from collections import Counter, ChainMap


class StyleMap:
    def __init__(self, mapping, counter: Counter = None):
        if not isinstance(mapping, ChainMap):
            mapping = ChainMap(mapping)
        if counter is None:
            counter = Counter()

        self.mapping = mapping
        self.counter = counter

    def merge(self, mapping, *, reset: bool = False):
        mapping = self.mapping.new_child(mapping)
        counter = Counter() if reset else self.counter
        return self.__class__(mapping, counter)

    def copy(self, *, reset: bool = True):
        mapping = self.mapping
        counter = Counter() if reset else self.counter
        return self.__class__(mapping, counter)

    def __repr__(self):
        cname = self.__class__.__name__
        return f"{cname}({self.mapping!r}, {self.counter!r})"

    def get_setting(self, key: str, default=None, *, prefix=None):
        if prefix:
            keys = [f"{p}.{key}" for p in prefix.split(":")]
        else:
            keys = [key]

        for key in keys:
            value = self.mapping.get(key)
            if isinstance(value, list) and len(value):
                count = self.counter[key]
                value = value[count % len(value)]
                self.counter[key] += 1
            if value is not None:
                return value

        return default


    def get_settings(self, prefix=None, **kwargs):
        for key, value in kwargs.items():
            value = self.get_setting(key, default=value, prefix=prefix)
            kwargs[key] = value

        return kwargs 


In [21]:

stylesheet = {
    "candlesticks.up.color": "black",
    "candlesticks.down.color": "red",
    "sma.color": ["blue", "green", "red"],
    "rsi.color": "black",
    "rsi.overbought.color": "grey",
    "rsi.oversold.color": "grey",
    "rsi.yticks.linewidth": 1.0,
    "rsi.yticks.linestyle": ".",
    "macd.color": None,
    "macdsignal.color": None,
    "*.overbought.color": "blue",
    "*.oversold.color": "blue",
    "yticks.linewidth": 1.0,
    "yticks.linestyle": None,
}


stylemap = StyleMap(stylesheet)

print(stylemap)

StyleMap(ChainMap({'candlesticks.up.color': 'black', 'candlesticks.down.color': 'red', 'sma.color': ['blue', 'green', 'red'], 'rsi.color': 'black', 'rsi.overbought.color': 'grey', 'rsi.oversold.color': 'grey', 'rsi.yticks.linewidth': 1.0, 'rsi.yticks.linestyle': '.', 'macd.color': None, 'macdsignal.color': None, '*.overbought.color': 'blue', '*.oversold.color': 'blue', 'yticks.linewidth': 1.0, 'yticks.linestyle': None}), Counter())


In [22]:
for key in ["rsi.color", "sma.color", "sma.color"]:
    value = stylemap.get_setting(key)
    print(key, repr(value))


rsi.color 'black'
sma.color 'blue'
sma.color 'green'


In [23]:
from collections import defaultdict

class Chart:
    def __init__(self, stylesheet):
        stylesheet = dict(stylesheet)
        self.stylemaps = defaultdict(lambda: StyleMap(stylesheet))

    def get_stylemap(self, ax):
        return self.stylemaps[ax]
    
    def get_settings(self, prefix, ax=None, **kwargs):
        return self.stylemaps[ax].get_settings()


chart = Chart(stylesheet=stylesheet)
chart.get_stylemap(None)

StyleMap(ChainMap({'candlesticks.up.color': 'black', 'candlesticks.down.color': 'red', 'sma.color': ['blue', 'green', 'red'], 'rsi.color': 'black', 'rsi.overbought.color': 'grey', 'rsi.oversold.color': 'grey', 'rsi.yticks.linewidth': 1.0, 'rsi.yticks.linestyle': '.', 'macd.color': None, 'macdsignal.color': None, '*.overbought.color': 'blue', '*.oversold.color': 'blue', 'yticks.linewidth': 1.0, 'yticks.linestyle': None}), Counter())